## Visualize LiDAR Scattered Point Elevation Data

This notebook uses datashader to visualize LiDAR elevation data from [the Puget Sound LiDAR consortium](http://pugetsoundlidar.ess.washington.edu/), a source of LiDAR data for the Puget Sound region of Washington, U.S.

The data are downloaded in 7z archives, then unzipped and visualized below.

## Setup

The following cell only needs to be run once on your new datashader conda environment.  The commands will:

 * Run the `download_sample_data.py` script to download LiDAR from [Puget Sound LiDAR consortium](http://pugetsoundlidar.ess.washington.edu)
 * Install `pylzma` for 7zip archive reading (the LiDAR files are .7z 7zip compressed files)
 * Make sure you're runninng `jupyter notebook` from the `examples` directory of your `datashader` clone.

In [ ]:
! cd datashader/examples && python download_sample_data.py
! cd data/pylzma-0.4.8/ && python setup.py install

*Grid_Coordinate_System_Name*: State Plane Coordinate System

*State_Plane_Coordinate_System*: SPCS_Zone_Identifier Washington North, FIPS 4601

*Lambert_Conformal_Conic*:
 * Standard_Parallel: 47.500000
 * Standard_Parallel: 48.733333
 * Longitude_of_Central_Meridian: -120.833333
 * Latitude_of_Projection_Origin: 47.000000
 * False_Easting: 1640416.666667
 * False_Northing: 0.000000
    
http://www.spatialreference.org/ref/esri/102348/

Washington State Plane North - FIPS 4601

In [ ]:
import sys
sys.path.append('data')
import glob
import os

from bokeh.models import WMTSTileSource
from bokeh.models import WMTSTileSource
from cartopy import crs as ccrs
from dask.distributed import Client
from holoviews.operation import datashade
from pyproj import Proj, transform
import dask.dataframe as dd
import datashader as ds
import geoviews as gv
import geoviews as gv
import holoviews as hv
import numpy as np
import rasterio as rio
client = Client()

In [ ]:
lidar_7z_dir = 'data/puget_sound_lidar-2/BE_ASCII'
lidar_gnd_dir = 'data/puget_sound_lidar-2-gdb/BE_ASCII'

def gnd_name(fname):
    if not os.path.exists(lidar_gnd_dir):
        os.makedirs(lidar_gnd_dir)
    return os.path.join(lidar_gnd_dir, os.path.basename(fname))

def unzip_7z(fname):
    from py7zlib import Archive7z
    try:
        arc = Archive7z(open(fname, 'rb'))
    except:
        print('FAILED ON 7Z', fname) # q47122e3114.7z has formatting issues
        return
    fnames = arc.filenames
    files = arc.files
    for fn, fi in zip(fnames, files):
        gnd = gnd_name(fn)
        if os.path.exists(gnd):
            yield gnd
        else:
            print('Extract', fname)
            with open(gnd, 'wb') as f2:
                f2.write(fi.read())
            yield gnd


In [ ]:
def unzip_all(lidar_7z_files):
    lidar_gnd_files = []
    for fname_7z in lidar_7z_files:
        lidar_gnd_files.extend(unzip_7z(fname_7z))
    return lidar_gnd_files


In [ ]:
lidar_7z_files = glob.glob(os.path.join(lidar_7z_dir, '*7z'))
lidar_gnd_files = unzip_all(lidar_7z_files)[:4]
lidar_gnd_files

In [ ]:
df_raw = dd.read_csv(lidar_gnd_files)

In [ ]:
df_raw.head()

In [ ]:
FT_2_M = 0.3048
washington_state_plane = Proj(init='epsg:2855')
web_mercator = Proj(init='epsg:3857')
    
def convert_coords(ddf):
    lon, lat = transform(washington_state_plane, web_mercator,ddf.X.values * FT_2_M, ddf.Y.values * FT_2_M)
    ddf['meterswest'], ddf['metersnorth'] = lon, lat 
    ddf2 = ddf[['meterswest', 'metersnorth', 'Z']].copy()
    del ddf
    return ddf2

import dask
import pandas as pd
@dask.delayed
def read_gnd(fname):
    return convert_coords(pd.read_csv(fname))

In [ ]:
df = dd.from_delayed([read_gnd(f) for f in lidar_gnd_files])
df.head()

In [ ]:

hv.notebook_extension('bokeh', width=95)

%opts Overlay [width=800 height=455 xaxis=None yaxis=None show_grid=False] 
%opts Shape (fill_color=None line_width=1.5) [apply_ranges=False] 
%opts Points [apply_ranges=False] WMTS (alpha=0.5) NdOverlay [tools=['tap']]

From the metadata for this Puget Sound Lidar Consortium data set:

```
West_Bounding_Coordinate: -123.12157
East_Bounding_Coordinate: -121.78725
North_Bounding_Coordinate: 47.94147
South_Bounding_Coordinate: 47.35807
```

In [ ]:
kdims = ['meterswest', 'metersnorth',]
min_x, min_y = web_mercator(-122.6, 47.4)   # From the data set metadata, but could be df.min() or .max() 
max_x, max_y = web_mercator(-122, 47.8) 
dataset = gv.Dataset(df, kdims=kdims, vdims=['Z'])
x_range, y_range = ((min_x, max_x), (min_y, max_y))
shade_defaults = dict(x_range=x_range, y_range=y_range, x_sampling=12, y_sampling=12, width=800, height=455)
tri = hv.Points(dataset, kdims=kdims, vdims=['Z'])
shaded = datashade(tri, **shade_defaults)

In [ ]:
tiles = gv.WMTS(WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'))

In [ ]:
shaded * tiles